In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
import sys, importlib
import xlwings as xw
import pandas as pd

In [23]:
# importlib.reload(sys.modules['pages.elements'])
# importlib.reload(sys.modules['pages.pages'])
importlib.reload(sys.modules['scripts.preencher'])
from scripts.inicializar import Inicializar
from scripts.preencher import Preencher
# from pages.pages import PageFatura
# from pages.elements import *

In [26]:
browser = Firefox()

pagina = Inicializar(
    browser,
    'https://guardiaov4.seplag.ce.gov.br/auth',
    'credenciais.txt'
).run()

In [24]:
# pagina = PageFatura(browser)
p = Preencher(browser, 'data/SPS-10-2021.xls', intervalo=(11, 23))

In [31]:
# p.dados.iloc[0]
pagina.terceirizados.carregar_funcionarios(p.dados)

In [34]:
p.run()

WebDriverException: Message: Failed to write request to stream


In [7]:
pagina.terceirizados.carregar_funcionarios(p.dados)

In [ ]:
p._obter_funcionarios()

In [5]:
wb = xw.Book('data\misto-06-2021.xlsx')
sheet = wb.sheets['Plan SPG']

In [ ]:
colunas = ['Nome', 'Dias Trabalhados', 'Salario Base', 
    'Salario Total', 'Adicional', 'Adicional Nortuno', 'Reserva', 
    'Encargos', 'Insalibridade', 'Periculosidade', 'Outros', 
    'Vale Transporte', 'Vale Refeicao', 'Taxa', 'Cesta', 'Farda', 
    'Municao', 'Seguro Vida', 'Supervisao', 'IJD', 'IJN', 'Tributos', 
    'Insumos', 'Equipamento', 'Plano Saude', 'Hora Extra', 
    'DSR', 'Extra Encargos',  'Extra Taxa',  'Extra',  'Viagem Tributos', 
    'Diarias',  'Passagem',  'Viagem', 'Viagem Taxa',  'Viagem Tributos'
]

df = pd.DataFrame(columns=colunas)

for i in range(11, 53):
    # 11:52
    valores_func = []
    valores_func += [sheet[f'A{i}'].value]
    valores_func += [sheet[f'C{i}'].value]
    valores_func += [sheet[f'E{i}'].value]
    valores_func += [sheet[f'R{i}'].value]
    valores_func += [sheet[f'G{i}'].value]
    valores_func += [0.] * 2
    valores_func += [sheet[f'H{i}'].value]
    valores_func += [0.]
    valores_func += [sheet[f'F{i}'].value]
    valores_func += [0.]
    valores_func += sheet[f'K{i}:L{i}'].value
    valores_func += [sheet[f'J{i}'].value]
    valores_func += sheet[f'M{i}:N{i}'].value
    valores_func += [0.] * 5
    valores_func += [sheet[f'P{i}'].value]
    valores_func += [0.] * 2
    valores_func += [sheet[f'O{i}'].value]
    valores_func += [0.] * 11
    s = pd.Series(valores_func, index=colunas)
    df = df.append(s, ignore_index=True)
# valores_func
df = df.fillna(0.)

for i in range(df.shape[0]):
    if df.iloc[i]['Salario Total'] == 0.:
        for c in df.columns[2:]:
            df.at[i, c] = 0.

# df.iloc[36]
df.head()

In [ ]:
wb = xw.Book('data/SDC-11-2021-01.xls')
sheet = wb.sheets['Plan SPG']

In [ ]:
campos = ['CPF', 'Nome', 'Dias Trabalhados', 'Salario Base', 
        'Salario Total', 'Adicional', 'Adicional Nortuno', 'Reserva', 
        'Encargos', 'Insalibridade', 'Periculosidade', 'Outros', 
        'Vale Transporte', 'Vale Refeicao', 'Taxa', 'Cesta', 'Farda', 
        'Municao', 'Seguro Vida', 'Supervisao', 'IJD', 'IJN', 'Tributos', 
        'Insumos', 'Equipamento', 'Plano Saude', 'Qtd Hora Extra', 
        'Valor Hora Extra', 'DSR', 'Hora Encargos',  'Hora Taxa', 'Hora Tributos', 
        'Qtd Diarias',  'Passagem',  'Viagem', 'Viagem Taxa',  'Viagem Tributos'
    ]

df = pd.DataFrame(columns=campos)

for i in range(10, 15):
    valores_func = []
    valores_func += [sheet[f'R{i}'].value] # CPF
    valores_func += [sheet[f'A{i}'].value] # Nome
    valores_func += [sheet[f'C{i}'].value] # Dias
    valores_func += [sheet[f'E{i}'].value] # Salario Base
    valores_func += [sheet[f'N{i}'].value] # Salario Total
    valores_func += [0.] * 3
    valores_func += [sheet[f'F{i}'].value] # Encargos
    valores_func += [0.] * 4
    valores_func += [sheet[f'I{i}'].value] # Vale Alimentacao
    valores_func += [sheet[f'H{i}'].value] # Taxa
    valores_func += [sheet[f'J{i}'].value] # Cesta
    valores_func += [0.] * 6
    valores_func += [sheet[f'L{i}'].value] # Tributos
    valores_func += [0.] * 2
    valores_func += [sheet[f'K{i}'].value]
    valores_func += [0.] * 11
    s = pd.Series(valores_func, index=campos)
    df = df.append(s, ignore_index=True)
# valores_func
df = df.fillna(0.)

for i in range(df.shape[0]):
    if df.iloc[i]['Salario Total'] == 0.:
        for c in df.columns[2:]:
            df.at[i, c] = 0.

df = df.set_index('CPF')
df.head()